In [2]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
from pymilvus import connections, utility, FieldSchema, CollectionSchema, DataType, Collection

# --- 1. 配置 ---
# 待处理的文件名 (请确保该文件在当前目录下，或者直接在此处填入完整的绝对路径)
FILE_NAME = "F:\\福卡\\福卡文章切片.xlsx"

# 获取绝对路径 (如果文件就在当前脚本同级目录下)
# 如果文件在其他位置，请直接修改为: SOURCE_DATA_FILE = r"D:\你的文件夹\福卡文章切片.xlsx"
SOURCE_DATA_FILE = os.path.abspath(FILE_NAME)

# Milvus 配置
MILVUS_HOST = "192.168.16.138"
MILVUS_PORT = "19530"
COLLECTION_NAME = "report_analysis1"
ID_FIELD = "chunk_id"
VECTOR_FIELD = "vector"
TEXT_FIELD = "text_content"
MODEL_NAME = 'all-mpnet-base-v2'
EMBEDDING_DIM = 768

# --- 2. 数据加载函数 (针对 Excel) ---
def load_data_from_excel(file_path):
    """
    读取 Excel 文件并提取指定列的数据
    """
    print(f"正在读取文件: {file_path}")
    
    if not os.path.exists(file_path):
        print(f"错误: 文件不存在 -> {file_path}")
        return None

    try:
        # 使用 openpyxl 引擎读取 xlsx 文件
        df = pd.read_excel(file_path, engine='openpyxl')

        # 检查是否存在目标列
        target_column = "切片"
        
        #以此去除列名两端的可能存在的空格，防止因为 ' 切片 ' 这种小问题读取失败
        df.columns = df.columns.str.strip() 

        if target_column not in df.columns:
            print(f"错误: Excel 文件中未找到列名 '{target_column}'。")
            print(f"现有列名: {list(df.columns)}")
            return None

        # 提取数据：
        # 1. 选中"切片"列
        # 2. dropna() 去除空行
        # 3. astype(str) 确保内容转换为字符串
        chunks = df[target_column].dropna().astype(str).tolist()
        
        # 过滤掉内容过短的切片 (比如只有标点或几个字的)
        chunks = [c.strip() for c in chunks if len(c.strip()) > 5]
        
        print(f"读取完成，共提取到 {len(chunks)} 条有效切片数据。")
        return chunks

    except Exception as e:
        print(f"读取 Excel 文件时发生异常: {e}")
        return None

# --- 3. 主流程 ---
def main():
    print(f"--- 开始处理 ---")
    print(f"目标文件绝对路径: {SOURCE_DATA_FILE}")

    # 1. 加载数据
    chunks = load_data_from_excel(SOURCE_DATA_FILE)
    
    if not chunks:
        print("未获取到有效数据，程序终止。")
        return

    # 预览数据
    print("\n--- 数据预览 ---")
    for i, chunk in enumerate(chunks[:3]):
        print(f"切片 {i+1}: {chunk[:50]}...")

    # 2. 生成向量
    print(f"\n正在加载嵌入模型 '{MODEL_NAME}'...")
    model = SentenceTransformer(MODEL_NAME)
    
    print(f"正在生成向量 (共 {len(chunks)} 条)...")
    embeddings = model.encode(chunks, show_progress_bar=True)
    
    # 3. 存入 Milvus
    print(f"\n正在连接 Milvus ({MILVUS_HOST}:{MILVUS_PORT})...")
    try:
        connections.connect("default", host=MILVUS_HOST, port=MILVUS_PORT)
    except Exception as e:
        print(f"连接 Milvus 失败: {e}")
        return
    
    # 如果集合存在，先删除重建 (根据需求决定是否保留)
    if utility.has_collection(COLLECTION_NAME):
        print(f"集合 '{COLLECTION_NAME}' 已存在，正在删除重建...")
        utility.drop_collection(COLLECTION_NAME)
        
    fields = [
        FieldSchema(name=ID_FIELD, dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name=VECTOR_FIELD, dtype=DataType.FLOAT_VECTOR, dim=EMBEDDING_DIM),
        # 设置最大长度，防止某些超长切片导致插入失败
        FieldSchema(name=TEXT_FIELD, dtype=DataType.VARCHAR, max_length=65535) 
    ]
    
    schema = CollectionSchema(fields, description="Report Analysis Chunks from Excel")
    collection = Collection(name=COLLECTION_NAME, schema=schema)
    
    # 插入数据
    # 注意：insert 的列表顺序必须与 Schema 中非 auto_id 字段的定义顺序一致
    # Schema: [ID(自动), VECTOR, TEXT] -> Insert: [embeddings, chunks]
    print(f"正在向 Milvus 插入 {len(chunks)} 条数据...")
    collection.insert([embeddings, chunks])
    
    print("正在等待数据写入磁盘 (Flush)...")
    collection.flush() 

    # 创建索引
    print("正在创建索引...")
    collection.create_index(
        VECTOR_FIELD, 
        {"metric_type": "L2", "index_type": "IVF_FLAT", "params": {"nlist": 128}}
    )
    
    # 加载集合
    collection.load()
    
    print(f"\n成功存入 {collection.num_entities} 条数据到 Milvus！")

if __name__ == "__main__":
    main()

--- 开始处理 ---
目标文件绝对路径: F:\福卡\福卡文章切片.xlsx
正在读取文件: F:\福卡\福卡文章切片.xlsx
读取完成，共提取到 671 条有效切片数据。

--- 数据预览 ---
切片 1: 〖特别报告〗  化解当下世界经济衰退的根本之道
——基础设施建设
摘要：人类历史遭逢二战以来最为严重...
切片 2: “此次经济衰退的范围和速度尚无现代先例，较之以往任何衰退都要严重得多。我们看到经济活动和就业率急剧下...
切片 3: 古语云“以史为鉴，方能知兴替”，人类经济史并不乏在遭遇经济萧条时冲破至暗时刻的先例，面对此轮史无前例...

正在加载嵌入模型 'all-mpnet-base-v2'...
正在生成向量 (共 671 条)...


Batches: 100%|██████████| 21/21 [01:21<00:00,  3.87s/it]



正在连接 Milvus (192.168.16.138:19530)...
正在向 Milvus 插入 671 条数据...
正在等待数据写入磁盘 (Flush)...
正在创建索引...

成功存入 671 条数据到 Milvus！
